# BERT Extractive Summarizer

## Import and load

In [1]:
!pip install bert-extractive-summarizer
!pip install --upgrade datasets
!pip install py7zr
!pip install spacy
!pip install transformers
!pip install neuralcoref
!pip install sentencepiece
!pip install rouge

!python -m spacy download en_core_web_md

     |████████████████████████████████| 2.3MB 3.8MB/s 
     |████████████████████████████████| 901kB 12.9MB/s 
     |████████████████████████████████| 3.3MB 18.9MB/s 
     |████████████████████████████████| 235kB 3.9MB/s 
     |████████████████████████████████| 112kB 18.8MB/s 
     |████████████████████████████████| 245kB 19.9MB/s 
     |████████████████████████████████| 71kB 2.6MB/s 
     |████████████████████████████████| 122kB 13.1MB/s 
     |████████████████████████████████| 358kB 37.5MB/s 
     |████████████████████████████████| 2.2MB 30.0MB/s 
     |████████████████████████████████| 1.9MB 29.9MB/s 
     |████████████████████████████████| 286kB 3.9MB/s 
     |████████████████████████████████| 133kB 24.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 7.6MB 18.4MB/s 
ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.2MB 3.9M

In [2]:
from datasets import load_dataset
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler
import pandas as pd

from rouge import Rouge
rouge = Rouge()

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:00<00:00, 42827823.90B/s]


In [3]:
dataset = load_dataset('samsum')

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


In [4]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Preprocess the data

In [5]:
test_df = pd.DataFrame(data={"dialogue": dataset["test"][:]['dialogue'], "summary": dataset["test"][:]['summary']})

In [6]:
def removeNewline(text):
    return text.replace('\r', ' ').replace('\n', '')

In [7]:
test_df.dialogue = test_df.dialogue.apply(lambda x: removeNewline(x))
test_df.summary = test_df.summary.apply(lambda x: removeNewline(x))

## Run the model

In [8]:
test_df["results"] = test_df.dialogue.apply(lambda x: model(x))

In [9]:
test_df.results

0      Hannah: Hey, do you have Betty's number?Amanda...
1      And shows how Americans see Russian ;) Rob: An...
2      Lenny: Babe, can you help me with something? B...
3      Will: hey babe, what do you want for dinner to...
4      Ollie: Hi , are you in Warsaw Jane: yes, just ...
                             ...                        
814    Alex: Were you able to attend Friday night's b...
815    Jamilla: remember that the audition starts at ...
816    Marta: <file_gif> Marta: Sorry girls, I clicke...
817    Cora: Have you heard how much fuss British med...
818    Rachel: <file_other> Rachel: Top 50 Best Films...
Name: results, Length: 819, dtype: object

In [10]:
test_df.results[0]

"Hannah: Hey, do you have Betty's number?Amanda: Lemme checkHannah: <file_gif>Amanda: Sorry, can't find it."

In [11]:
test_df.summary[0]

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [12]:
test_df.dialogue[0]

"Hannah: Hey, do you have Betty's number?Amanda: Lemme checkHannah: <file_gif>Amanda: Sorry, can't find it.Amanda: Ask LarryAmanda: He called her last time we were at the park togetherHannah: I don't know him wellHannah: <file_gif>Amanda: Don't be shy, he's very niceHannah: If you say so..Hannah: I'd rather you texted himAmanda: Just text him 🙂Hannah: Urgh.. AlrightHannah: ByeAmanda: Bye bye"

In [13]:
test_clean_df = test_df[test_df["results"] != '']

## Measure

In [14]:
rouge.get_scores(test_clean_df.results.to_list(), test_clean_df.summary.to_list(), avg=True)

{'rouge-1': {'f': 0.18042705015226385,
  'p': 0.169724307795239,
  'r': 0.24147231864197902},
 'rouge-2': {'f': 0.06480524254141942,
  'p': 0.06309074371137323,
  'r': 0.08485573892374126},
 'rouge-l': {'f': 0.17504957465548734,
  'p': 0.16459860847832455,
  'r': 0.22404792247207642}}